In [128]:
import os
import shutil
# import pandas as pd
from tqdm import tqdm

In [148]:
def split_directory_by_labels(src_directory, dest_directory, label_map, label_map_names, reset):
    if reset and os.path.exists(dest_directory):
        shutil.rmtree(dest_directory)
        
    if not os.path.isdir(dest_directory):
        os.mkdir(dest_directory)
        
    for i in label_map_names:
        if not os.path.isdir(label_map_names[i]):
            os.mkdir(os.path.join(dest_directory, label_map_names[i]))
            
    
    src_files = os.listdir(src_directory)
    
    print("Number of files in from directory: ", len(src_files))
    
    for filename in tqdm(src_files):
        label = label_map[filename[:-4]]
        
        src_file_path = os.path.join(src_directory, filename)
        
        cur_dest_directory = os.path.join(dest_directory, label_map_names[label])
        dest_filepath = os.path.join(cur_dest_directory, filename)
        
        if not os.path.exists(dest_filepath):
            shutil.copyfile(src_file_path, dest_file_path)

    print("Split directory by labels complete")      
            
            
def train_validation_test_split(master_dir, train_dir, validation_dir, test_dir, train_percent, validation_percent, test_percent, copy):        
    
    dir_list = os.listdir(master_dir)
    
    train_count=0
    validation_count=0
    test_count=0
    
    total_fail_count = 0
    
    for directory in dir_list:
        src_subdir = os.path.join(master_dir, directory)
        
        if train_dir:
            dest_train_subdir = os.path.join(train_dir, directory)
        dest_validation_subdir = os.path.join(validation_dir, directory)
        dest_test_subdir = os.path.join(test_dir, directory)
        
        if train_dir and not os.path.isdir(dest_train_subdir):
            os.mkdir(dest_train_subdir)
        if not os.path.isdir(dest_validation_subdir):
            os.mkdir(dest_validation_subdir)
        if not os.path.isdir(dest_test_subdir):
            os.mkdir(dest_test_subdir)
            
        
        
        src_subdir_files = os.listdir(src_subdir)
        count_files = len(src_subdir_files)

        if train_dir:
            train_count = count_files*(train_percent/100)
            validation_count = train_count+count_files*(validation_percent/100)
            test_count = validation_count+count_files*(test_percent/100)
        else:
            validation_count = count_files*(validation_percent/100)
            test_count = validation_count+count_files*(test_percent/100)
            
        
        print("Moving file from: %s, total files: %d, train count: %d, validation count: %d, test count: %d" % (src_subdir, count_files, train_count, validation_count, test_count))
        
        i = 0
        fail_count = 0
        
#         for filename in tqdm(src_subdir_files):
        for filename in src_subdir_files:
            try:
#                 print(filename)
                src_filepath = os.path.join(src_subdir, filename)
    
                dest_filepath=""
    
                if train_dir:
                    if i<train_count:
                        dest_filepath = os.path.join(dest_train_subdir, filename)
                    if i>=train_count and i <validation_count:
                        dest_filepath = os.path.join(dest_validation_subdir, filename)
                    else:
                        dest_filepath = os.path.join(dest_test_subdir, filename)
                else:
                    if i<validation_count:
                        dest_filepath = os.path.join(dest_validation_subdir, filename)
                    else:
                        dest_filepath = os.path.join(dest_test_subdir, filename)
                        
                    
                if copy:
                    shutil.copyfile(src_filepath, dest_filepath)
                i+=1
            except:
                fail_count+=1
                print("Exception for %s: filename: %s"% (i, filename))
    
        total_fail_count+=fail_count                      
        print("Move Complete total prev: %d, total_now(src): %d, total train: %d, total validation: %d, total test: %d Failed: %d" % (count_files, len(os.listdir(src_subdir)), len(os.listdir(dest_train_subdir)), len(os.listdir(dest_validation_subdir)), len(os.listdir(dest_test_subdir)), fail_count))
              
    print("Move Complete, Failed: %d" % (total_fail_count))
    

    
def train_validation_test_split_main(master_dir, master_dir_type, train_dir, validation_dir, test_dir, train_percent, validation_percent, test_percent, copy):    
    if train_dir and not os.path.isdir(train_dir):
        os.mkdir(train_dir)
    if not os.path.isdir(validation_dir):
        os.mkdir(validation_dir)
    if not os.path.isdir(test_dir):
        os.mkdir(test_dir)
        
    if master_dir_type=="train":
        copy_fraction_file(master_dir, train_dir, validation_dir, test_dir, train_percent, validation_percent, test_percent, copy)
    
    elif master_dir_type=="test":
        copy_fraction_file(master_dir, None, validation_dir, test_dir, 0, validation_percent, test_percent, copy)


        
def move_file_from_dir_to_dir(from_dir, to_dir, move_percent, copy):    
    if not os.path.isdir(to_dir):
        os.mkdir(to_dir)
    
    dir_list = os.listdir(from_dir)
    
    tot_fail=0
    
    for directory in dir_list:
        
        from_subdir = os.path.join(from_dir, directory)
        to_subdir = os.path.join(to_dir, directory)
        
        if not os.path.isdir(to_subdir):
            os.mkdir(to_subdir)
        
        
        from_subdir_files = os.listdir(from_subdir)
        

        count_files = len(from_subdir_files)
        print(from_subdir, count_files)

        move_count = count_files*(move_percent/100)
        
        print("Moving file from: %s,  to: %s, total files: %d, moving: %d" % (from_subdir, to_subdir, count_files, move_count))
        
        i = 0
        fail = 0
        
        for filename in tqdm(from_subdir_files):
            try:
#                 print(filename)
                from_filepath = os.path.join(from_subdir, filename)
                
                if i<move_count:
                    shutil.copyfile(from_filepath, os.path.join(to_subdir, filename))
                    i+=1
                else:
                    break
            except:
                fail+=1
                print("Exception for %s: filename: %s"% (i, filename))
        
        tot_fail+=fail                      
        print("Move Complete total prev: %d, total_now(from): %d, total_to: %d, Failed: %d" % (count_files, len(os.listdir(from_subdir)), len(os.listdir(to_subdir)), fail))
              
    print("Move Complete, Failed: %d" % (tot_fail))
    
def move_file_from_subdir_to_subdir(from_dir, to_dir, move_count):         
    from_subdir_files = os.listdir(from_subdir)
    count_files = len(from_subdir_files)        
    i = 0
    fail = 0
        
    for filename in tqdm(from_subdir_files):
        try:
            if i<move_count:
                shutil.move(os.path.join(from_dir, filename), os.path.join(to_subdir, filename))
                i+=1
            else:
                break
        except:
            fail+=1
            print("Exception for %s: filename: %s"% (i, filename))
                     
    print("Move Complete, Failed: %d" % (fail))


def balance_train_validation_test_set(master_dir, train_dir_type, train_dir, validation_dir, test_dir, train_percent, validation_percent, test_percent):            
    dir_list = os.listdir(master_dir)
    
    train_count=0
    validation_count=0
    test_count=0
    
    total_fail_count = 0
    
    for directory in dir_list:
        if train_dir:
            train_subdir = os.path.join(train_dir, directory)
        validation_subdir = os.path.join(validation_dir, directory)
        test_subdir = os.path.join(test_dir, directory)
        
        
        current_train_count = os.listdir(train_subdir)
        current_validation_count = os.listdir(validation_subdir)
        current_test_count = os.listdir(test_subdir)
        

        if train_dir:
            total_count = train_count + validation_count + test_count
            
            expected_train_count = total_count*(train_percent/100)
            expected_validation_count = total_count*(validation_percent/100)
            expected_test_count = total_count*(test_percent/100)
            
        else:
            total_count = validation_count + test_count
            
            expected_validation_count = total_count*(validation_percent/100)
            expected_test_count = total_count*(test_percent/100)
            
        
        if current_train_count<expected_train_count and current_validation_count>expected_validation_count:
            extra = current_validation_count-expected_validation_count
            need = expected_train_count-current_train_count
            if need>extra:
                move=extra
            else:
                move=need
            move_file_from_subdir_to_subdir(validation_subdir, train_subdir, move)
            current_train_count = current_train_count+move
            current_validation_count = current_validation_count-move
            
        if current_train_count<expected_train_count and current_test_count>expected_test_count:
            extra = current_test_count-expected_test_count
            need = expected_train_count-current_train_count
            if need>extra:
                move = extra
            else:
                move = need
            move_file_from_subdir_to_subdir(test_subdir, train_subdir, move)
            current_train_count = current_train_count+move
            current_test_count = current_test_count-move
            
        if current_validation_count<expected_validation_count and current_test_count>expected_test_count:
            extra = current_test_count-expected_test_count
            need = expected_validation_count-current_validation_count
            if need>extra:
                move = extra
            else:
                move = need
            move_file_from_subdir_to_subdir(test_subdir, validation_subdir, move)
            current_validation_count = current_validation_count+move
            current_test_count = current_test_count-move
            
        if current_validation_count<expected_validation_count and current_train_count>expected_train_count:
            extra = current_train_count-expected_train_count
            need = expected_validation_count-current_validation_count
            if need>extra:
                move = extra
            else:
                move = need
            move_file_from_subdir_to_subdir(train_subdir, validation_subdir, move)
            current_validation_count = current_validation_count+move
            current_train_count = current_train_count-move
            
        
        
        if current_test_count<expected_test_count and current_validation_count>expected_validation_count:
            extra = current_validation_count-expected_validation_count
            need = expected_test_count-current_test_count
            if need>extra:
                move = extra
            else:
                move = need
            move_file_from_subdir_to_subdir(validation_subdir, test_subdir, move)
            current_test_count = current_test_count+move
            current_validation_count = current_validation_count-move
            
        if current_test_count<expected_test_count and current_train_count>expected_train_count:
            extra = current_train_count-expected_train_count
            need = expected_test_count-current_test_count
            if need>extra:
                move = extra
            else:
                move = need
            move_file_from_subdir_to_subdir(train_subdir, test_subdir, move)
            current_test_count = current_test_count+move
            current_train_count = current_train_count-move
            
            
        
        current_train_count = os.listdir(train_subdir)
        current_validation_count = os.listdir(validation_subdir)
        current_test_count = os.listdir(test_subdir)
        print("current sub-directory: %s: current_train_count:%d, current_train_count: %d, current_train_count:%d"%(directory, current_train_count, current_validation_count, current_test_count))
    print("completed")

In [149]:
src_train_directory_all = "data\\input\\train"
src_train_directory = "data\\input\\train_new"
src_test_directory_all = "data\\input\\test"
src_validation_directory = "data\\input\\test_new"

dest_train_directory = "data\\input\\train_final"
dest_validation_directory = "data\\input\\validation_final"
dest_test_directory = "data\\input\\test_final"

In [131]:
train_labels = pd.read_csv("data/input/train_labels.csv")
test_labels = pd.read_csv("data/input/sample_submission.csv")

# train labels
labels_only = train_labels['label']

labels_set = set(labels_only)

num_labels = len(labels_set)

label_map_names = {0: "Normal", 1 : "Cancer"}
    

label_map={}
for i in range(len(train_labels['id'])):
    label_map[train_labels['id'][i]] = train_labels['label'][i]
    

# test labels
sample_label_map={}
for i in range(len(test_labels['id'])):
    sample_label_map[test_labels['id'][i]] = test_labels['label'][i]

From excel sheet create map for all the images mentioned in sheet

In [24]:
# split_directory_by_labels(src_train_directory_all, src_train_directory, label_map, label_map_names, reset=True)
# label_dir(test_from_directory, test_to_directory, sample_label_map, label_map_names, reset=True)

Number of files in from directory:  220025


100%|██████████| 220025/220025 [2:24:35<00:00, 36.78it/s]   


From labels saved in excel file create diferent directory for different class for a directory with multiple classes

In [144]:
master_dir=src_train_directory
master_dir_type = "train"
train_dir=dest_train_directory
# train_dir=None
validation_dir = dest_validation_directory
test_dir = dest_test_directory
train_percent = 60
validation_percent = 20
test_percent = 20    
copy=True


In [150]:
train_validation_test_split_main(master_dir, master_dir_type, train_dir, validation_dir, test_dir, train_percent, validation_percent, test_percent, copy)

Moving file from: data\input\train_new\Cancer, total files: 89117, train count: 53470, validation count: 71293, test count: 89117




  0%|          | 0/89117 [00:00<?, ?it/s]

  0%|          | 7/89117 [00:00<23:19, 63.68it/s]

  0%|          | 17/89117 [00:00<21:11, 70.06it/s]

  0%|          | 27/89117 [00:00<19:30, 76.14it/s]

  0%|          | 36/89117 [00:00<18:49, 78.86it/s]

  0%|          | 43/89117 [00:00<20:12, 73.49it/s]

  0%|          | 51/89117 [00:00<20:02, 74.09it/s]

  0%|          | 58/89117 [00:00<30:13, 49.10it/s]

  0%|          | 64/89117 [00:01<28:45, 51.62it/s]

  0%|          | 70/89117 [00:01<29:16, 50.71it/s]

  0%|          | 77/89117 [00:01<28:10, 52.68it/s]

  0%|          | 85/89117 [00:01<25:25, 58.35it/s]

  0%|          | 92/89117 [00:01<25:20, 58.57it/s]

  0%|          | 99/89117 [00:01<24:24, 60.77it/s]

  0%|          | 108/89117 [00:01<22:25, 66.15it/s]

  0%|          | 115/89117 [00:01<22:20, 66.38it/s]

  0%|          | 122/89117 [00:01<28:17, 52.42it/s]

  0%|          | 130/89117 [00:02<25:57, 57.15it/s]

  0%|          | 138/89117 [00:02<24:02, 61.67it/s]

  0%|          

  1%|          | 1091/89117 [00:20<32:50, 44.67it/s]

  1%|          | 1098/89117 [00:20<29:51, 49.13it/s]

  1%|          | 1107/89117 [00:20<26:32, 55.28it/s]

  1%|▏         | 1114/89117 [00:20<26:14, 55.89it/s]

  1%|▏         | 1121/89117 [00:20<30:27, 48.15it/s]

  1%|▏         | 1128/89117 [00:21<28:05, 52.20it/s]

  1%|▏         | 1136/89117 [00:21<26:18, 55.73it/s]

  1%|▏         | 1142/89117 [00:21<32:10, 45.57it/s]

  1%|▏         | 1148/89117 [00:21<31:43, 46.20it/s]

  1%|▏         | 1157/89117 [00:21<27:13, 53.84it/s]

  1%|▏         | 1165/89117 [00:21<25:10, 58.24it/s]

  1%|▏         | 1172/89117 [00:21<25:39, 57.13it/s]

  1%|▏         | 1179/89117 [00:21<26:55, 54.45it/s]

  1%|▏         | 1187/89117 [00:22<24:36, 59.54it/s]

  1%|▏         | 1195/89117 [00:22<22:46, 64.35it/s]

  1%|▏         | 1202/89117 [00:22<27:27, 53.36it/s]

  1%|▏         | 1210/89117 [00:22<25:04, 58.43it/s]

  1%|▏         | 1218/89117 [00:22<23:20, 62.76it/s]

  1%|▏         | 1225/89117 

  2%|▏         | 2202/89117 [00:40<22:58, 63.07it/s]

  2%|▏         | 2209/89117 [00:40<22:49, 63.46it/s]

  2%|▏         | 2217/89117 [00:40<22:08, 65.42it/s]

  2%|▏         | 2226/89117 [00:40<20:30, 70.61it/s]

  3%|▎         | 2235/89117 [00:40<19:21, 74.83it/s]

  3%|▎         | 2243/89117 [00:40<21:03, 68.76it/s]

  3%|▎         | 2252/89117 [00:40<19:50, 72.97it/s]

  3%|▎         | 2261/89117 [00:41<18:53, 76.62it/s]

  3%|▎         | 2270/89117 [00:41<19:14, 75.25it/s]

  3%|▎         | 2278/89117 [00:41<27:20, 52.92it/s]

  3%|▎         | 2285/89117 [00:41<29:14, 49.48it/s]

  3%|▎         | 2293/89117 [00:41<26:01, 55.59it/s]

  3%|▎         | 2300/89117 [00:41<27:56, 51.78it/s]

  3%|▎         | 2309/89117 [00:41<24:54, 58.08it/s]

  3%|▎         | 2319/89117 [00:42<22:42, 63.70it/s]

  3%|▎         | 2328/89117 [00:42<21:16, 67.99it/s]

  3%|▎         | 2338/89117 [00:42<19:36, 73.73it/s]

  3%|▎         | 2346/89117 [00:42<25:18, 57.15it/s]

  3%|▎         | 2354/89117 

  4%|▍         | 3347/89117 [00:59<24:58, 57.25it/s]

  4%|▍         | 3356/89117 [00:59<22:53, 62.42it/s]

  4%|▍         | 3364/89117 [00:59<21:29, 66.48it/s]

  4%|▍         | 3373/89117 [01:00<19:48, 72.12it/s]

  4%|▍         | 3382/89117 [01:00<18:42, 76.38it/s]

  4%|▍         | 3391/89117 [01:00<19:07, 74.68it/s]

  4%|▍         | 3399/89117 [01:00<19:03, 74.97it/s]

  4%|▍         | 3408/89117 [01:00<18:25, 77.56it/s]

  4%|▍         | 3417/89117 [01:00<17:44, 80.54it/s]

  4%|▍         | 3426/89117 [01:00<20:26, 69.88it/s]

  4%|▍         | 3434/89117 [01:00<20:55, 68.24it/s]

  4%|▍         | 3442/89117 [01:01<20:44, 68.86it/s]

  4%|▍         | 3450/89117 [01:01<20:26, 69.84it/s]

  4%|▍         | 3458/89117 [01:01<25:19, 56.38it/s]

  4%|▍         | 3465/89117 [01:01<24:32, 58.17it/s]

  4%|▍         | 3472/89117 [01:01<24:25, 58.44it/s]

  4%|▍         | 3480/89117 [01:01<23:08, 61.70it/s]

  4%|▍         | 3487/89117 [01:01<22:44, 62.77it/s]

  4%|▍         | 3495/89117 

...

 96%|█████████▌| 125894/130908 [35:25<00:58, 85.56it/s]

 96%|█████████▌| 125904/130908 [35:25<00:57, 87.16it/s]

 96%|█████████▌| 125914/130908 [35:25<01:02, 79.88it/s]

 96%|█████████▌| 125924/130908 [35:25<00:59, 83.17it/s]

 96%|█████████▌| 125933/130908 [35:25<00:59, 83.75it/s]

 96%|█████████▌| 125943/130908 [35:25<00:56, 87.64it/s]

 96%|█████████▌| 125953/130908 [35:25<00:55, 89.86it/s]

 96%|█████████▌| 125963/130908 [35:25<00:53, 92.23it/s]

 96%|█████████▌| 125973/130908 [35:25<00:56, 88.03it/s]

 96%|█████████▌| 125983/130908 [35:26<00:54, 90.02it/s]

 96%|█████████▌| 125993/130908 [35:26<00:54, 90.53it/s]

 96%|█████████▋| 126003/130908 [35:26<01:25, 57.64it/s]

 96%|█████████▋| 126011/130908 [35:26<01:19, 61.41it/s]

 96%|█████████▋| 126021/130908 [35:26<01:10, 68.85it/s]

 96%|█████████▋| 126030/130908 [35:26<01:06, 73.11it/s]

 96%|█████████▋| 126041/130908 [35:26<01:02, 77.61it/s]

 96%|█████████▋| 126052/130908 [35:27<00:57, 84.95it/s]

 96%|█████████▋| 126062/130908 

 97%|█████████▋| 127342/130908 [35:43<00:50, 70.49it/s]

 97%|█████████▋| 127351/130908 [35:43<00:48, 73.79it/s]

 97%|█████████▋| 127362/130908 [35:43<00:43, 80.96it/s]

 97%|█████████▋| 127372/130908 [35:43<00:43, 80.97it/s]

 97%|█████████▋| 127382/130908 [35:43<00:41, 85.62it/s]

 97%|█████████▋| 127393/130908 [35:43<00:38, 91.04it/s]

 97%|█████████▋| 127404/130908 [35:44<00:37, 93.99it/s]

 97%|█████████▋| 127414/130908 [35:44<00:36, 95.28it/s]

 97%|█████████▋| 127424/130908 [35:44<00:37, 92.11it/s]

 97%|█████████▋| 127434/130908 [35:44<00:37, 93.49it/s]

 97%|█████████▋| 127444/130908 [35:44<00:37, 91.75it/s]

 97%|█████████▋| 127455/130908 [35:44<00:36, 95.08it/s]

 97%|█████████▋| 127466/130908 [35:44<00:35, 98.10it/s]

 97%|█████████▋| 127476/130908 [35:44<00:34, 98.37it/s]

 97%|█████████▋| 127487/130908 [35:44<00:34, 100.01it/s]

 97%|█████████▋| 127498/130908 [35:45<00:36, 93.94it/s] 

 97%|█████████▋| 127508/130908 [35:45<00:40, 84.68it/s]

 97%|█████████▋| 127517/13090

 98%|█████████▊| 128793/130908 [36:00<00:21, 98.93it/s]

 98%|█████████▊| 128804/130908 [36:01<00:20, 100.30it/s]

 98%|█████████▊| 128815/130908 [36:01<00:20, 101.37it/s]

 98%|█████████▊| 128826/130908 [36:01<00:21, 99.04it/s] 

 98%|█████████▊| 128836/130908 [36:01<00:21, 97.67it/s]

 98%|█████████▊| 128846/130908 [36:01<00:21, 94.39it/s]

 98%|█████████▊| 128856/130908 [36:02<00:54, 37.86it/s]

 98%|█████████▊| 128864/130908 [36:02<00:46, 43.63it/s]

 98%|█████████▊| 128872/130908 [36:02<00:40, 49.81it/s]

 98%|█████████▊| 128880/130908 [36:02<00:37, 54.76it/s]

 98%|█████████▊| 128890/130908 [36:02<00:32, 62.56it/s]

 98%|█████████▊| 128900/130908 [36:02<00:29, 67.62it/s]

 98%|█████████▊| 128911/130908 [36:02<00:26, 75.57it/s]

 98%|█████████▊| 128920/130908 [36:02<00:25, 78.82it/s]

 98%|█████████▊| 128932/130908 [36:02<00:22, 86.29it/s]

 98%|█████████▊| 128942/130908 [36:03<00:21, 89.66it/s]

 99%|█████████▊| 128952/130908 [36:03<00:21, 91.88it/s]

 99%|█████████▊| 128963/1309

 99%|█████████▉| 130206/130908 [36:20<00:08, 87.62it/s]

 99%|█████████▉| 130217/130908 [36:21<00:07, 92.19it/s]

 99%|█████████▉| 130227/130908 [36:21<00:07, 92.62it/s]

 99%|█████████▉| 130237/130908 [36:21<00:12, 55.06it/s]

 99%|█████████▉| 130245/130908 [36:21<00:10, 60.66it/s]

100%|█████████▉| 130256/130908 [36:21<00:09, 69.43it/s]

100%|█████████▉| 130265/130908 [36:21<00:09, 70.19it/s]

100%|█████████▉| 130275/130908 [36:21<00:08, 74.95it/s]

100%|█████████▉| 130286/130908 [36:22<00:07, 82.70it/s]

100%|█████████▉| 130296/130908 [36:22<00:07, 86.02it/s]

100%|█████████▉| 130306/130908 [36:22<00:06, 86.91it/s]

100%|█████████▉| 130317/130908 [36:22<00:06, 92.40it/s]

100%|█████████▉| 130327/130908 [36:22<00:06, 93.01it/s]

100%|█████████▉| 130338/130908 [36:22<00:05, 96.62it/s]

100%|█████████▉| 130348/130908 [36:22<00:05, 96.93it/s]

100%|█████████▉| 130359/130908 [36:22<00:05, 99.00it/s]

100%|█████████▉| 130370/130908 [36:22<00:06, 89.49it/s]

100%|█████████▉| 130380/130908 

Move Complete total prev: 130908, total_now(src): 130908, total train: 0, total validation: 26182, total test: 104726 Failed: 0
Move Complete, Failed: 0


In [107]:
copy_fraction_file(train_from_directory2, train_to_directory, validate_to_directory, test_to_directory, train_frac=60, validation_frac=20, test_frac=20)

data\input\train_new\Cancer 89117
Moving file from: data\input\train_new\Cancer,  to: data\input\train_cat\Cancer, total files: 89117, moving: 53470


 60%|█████▉    | 53470/89117 [23:30<57:01, 10.42it/s]   

Move Complete total prev: 89117, total_now(from): 89117, total_to: 53471, Failed: 0
data\input\train_new\Normal 130908
Moving file from: data\input\train_new\Normal,  to: data\input\train_cat\Normal, total files: 130908, moving: 78544



  1%|          | 998/130908 [00:23<2:35:23, 13.93it/s]


  1%|          | 1237/130908 [00:54<4:49:07,  7.47it/s]


...

 57%|█████▋    | 74906/130908 [23:43<12:30, 74.66it/s]


 58%|█████▊    | 76193/130908 [24:02<11:40, 78.13it/s]


 59%|█████▉    | 77519/130908 [24:20<12:13, 72.77it/s]


 60%|█████▉    | 78538/130908 [24:45<10:51, 80.42it/s]

Move Complete total prev: 130908, total_now(from): 130908, total_to: 78545, Failed: 0
Move Complete, Failed: 0


In [ ]:
copy_fraction_file(train_from_directory2, test_to_directory, move_percent=50)

In [ ]:
# for directory in os.listdir(master_directory):
#     sub_directory = os.path.join(master_directory, directory)
#     sub_directory_files = os.listdir(sub_directory)
#     count_files = len(sub_directory_files)
#     print(directory, count_files)
#     train_count = count_files*.7
#     validate_count = train_count+count_files*.2
#     test_count = validate_count+count_files*.1
    
#     cat_train_dir = os.path.join(train_to_directory, directory)
#     cat_test_dir = os.path.join(test_to_directory, directory)
#     cat_validate_dir = os.path.join(validate_to_directory, directory)
    
#     if not os.path.isdir(cat_train_dir):
#         os.mkdir(cat_train_dir)
        
#     if not os.path.isdir(cat_validate_dir):
#         os.mkdir(cat_validate_dir)
    
#     if not os.path.isdir(cat_test_dir):
#         os.mkdir(cat_test_dir)
                 

    
#     from_dir_path = os.path.join(master_directory, directory)
#     from_dir_files= os.listdir(from_dir_path)
#     i = 0
#     for filename in from_dir_files:
#         try:
#             file_path_from = os.path.join(from_dir_path, filename)
#             if i<=train_count:
#                 shutil.copyfile(file_path_from, os.path.join(cat_train_dir, filename))
#             elif i<=validate_count:
#                 shutil.copyfile(file_path_from, os.path.join(cat_validate_dir, filename))
#             else:
#                 shutil.copyfile(file_path_from, os.path.join(cat_test_dir, filename))

#         except:
#             print("Exception for index: ", i, "Breed: ", label, ", File: ", file_path_from)
#         i+=1
